In [1]:
import pandas as pd 
import time
import traceback

from ast import literal_eval
import numpy as np
from datetime import datetime
from pymongo import MongoClient

import AllSample as ass
import Filter2 as fil2
import TopWords as top
import WordCount3 as woc3
import WordTF_IDF as tf_idf

from flask import Flask, render_template, request, jsonify

In [2]:
t0 = time.time()
client = MongoClient()
db = client.test_database
collection = db.test_collection

#reviewsMongo = db.reviews_mini
restaurantsMongo = db.restaurants_change
reviews_plainMongo = db.reviews_plain

#allDataLem = pd.DataFrame(list(reviewsMongo.find({})))
#t1 = time.time()
#print("Database converted in: " + str((t1-t0)/60))
#del allDataLem["_id"]
#allDataLem = allDataLem.astype({'stars': np.int8})
#t2 = time.time()
#print("allDataLem done in: " + str((t2-t0)/60))

restaurantNames = pd.DataFrame(list(restaurantsMongo.find({})))
del restaurantNames["_id"]
restaurantNames = restaurantNames.to_json(orient='records')
t3 = time.time()
print("Restaurants done in: " + str((t3-t2)/60))

reviewsPlain = pd.DataFrame(list(reviews_plainMongo.find({})))
del reviewsPlain["_id"]
reviewsPlain = reviewsPlain.astype({'stars': np.int8})
t4 = time.time()
print("Just reviews in: " + str((t4-t3)/60))

print("All done in: " + str((t4-t0)/60))

Database converted in: 1.2791733781496684
allDataLem done in: 1.4165433605511983
Restaurants done in: 0.006100304921468099
Just reviews in: 0.92900151014328
All done in: 2.3516451756159467


In [ ]:
lastDF = pd.DataFrame(index = ['A'], columns = ['new_name'])
app = Flask(__name__)

@app.route("/")
def index():
    return render_template("/index4.html")
    
@app.route("/AllSample")
def allSample():
    return render_template("AllSample3.html")

@app.route("/restaurantsNames", methods=['POST'])
def restaurantsNames():
    return jsonify(restaurantNames)

@app.route("/topWords", methods=['GET', 'POST'])
def endpointTW():    
    if request.method == 'POST':
        t0 = time.time()
        try:
            currentDF = allDataLem.copy()
            newName = request.form['newName']
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            filtered = fil2.main(currentDF, "", newName, rating, dateLow, dateUp)            
            if (filtered.empty):
                return "No data for those filters."        
            status = top.main(filtered["lemmas"])
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return str(status)
    else:
        return "Send a proper POST request!"
    
@app.route("/tf_idf", methods=['GET', 'POST'])
def endpointTF_IDF():    
    if request.method == 'POST':
        try:
            currentDF = allDataLem.copy()
            newName = request.form['newName']
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            granulation = request.form['granulation']
            if granulation == 'Year':
                currentDF = fil2.main(currentDF, "", newName, "", "", "")
                if (currentDF.empty):
                    return "No data for those filters."
                filtered0 = currentDF.copy()
                filtered1 = fil2.main(currentDF, "", "", "", dateLow, dateUp)            
                if (filtered1.empty):
                    return "No data for those time periods."         
                if (filtered1.equals(filtered0)):
                    return "Nothing to compare with."
                status = tf_idf.main(filtered0, filtered1, granulation)
            elif granulation == 'Month':
                currentDF = fil2.main(currentDF, "", newName, "", dateLow[0:5] + '01-01', dateUp[0:5] + '12-31')
                if (currentDF.empty):
                    return "No data for those filters."
                filtered0 = currentDF.copy()
                filtered1 = fil2.main(currentDF, "", "", "", dateLow, dateUp)            
                if (filtered1.empty):
                    return "No data for those time periods."            
                if (filtered1.equals(filtered0)):
                    return "Nothing to compare with."
                status = tf_idf.main(filtered0, filtered1, granulation)
            elif granulation == 'Rating':
                currentDF = fil2.main(currentDF, "", newName, "", dateLow, dateUp)
                if (currentDF.empty):
                    return "No data for those filters."
                filtered0 = currentDF.copy()
                filtered1 = fil2.main(currentDF, "", "", rating, "", "")            
                if (filtered1.empty):
                    return "No data for those ratings."         
                if (filtered1.equals(filtered0)):
                    return "Nothing to compare with."
                status = tf_idf.main(filtered0, filtered1, granulation)
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return str(status)
    else:
        return "Send a proper POST request!"
    
@app.route("/yearView")
def endpointYV():    
    return  render_template("/SingleRestaurant2.html")

@app.route("/allRestReviews")
def endpointARR():    
    return  render_template("/SingleRestaurant3.html")

@app.route("/plainReviews", methods=['GET', 'POST'])
def endpointRP():    
    if request.method == 'POST':
        t0 = time.time()
        try:
            plainCDF = reviewsPlain.copy()
            newName = request.form['newName']
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            filtered = fil2.main(plainCDF, "", newName, rating, dateLow, dateUp)            
            if (filtered.empty):
                return "No data for those filters."    
            status = list()
            
            if filtered.shape[0] >= 5:
                randomRow = np.random.choice(range(filtered.shape[0]), 5, replace=False)
                for i in randomRow:      
                    status.append(str(filtered["stars"].iloc[i]))
                    status.append(str(filtered["date"].iloc[i]))
                    status.append(str(filtered["text"].iloc[i]))
            else:
                for i in range(filtered.shape[0]):
                    status.append(str(filtered["stars"].iloc[i]))
                    status.append(str(filtered["date"].iloc[i]))
                    status.append(str(filtered["text"].iloc[i]))
        except:
            return "Python  error: " + traceback.format_exc().splitlines()[-1]
        return jsonify(status)
    else:
        return "Send a proper POST request!"
    
@app.route("/plainReviewsAll", methods=['GET', 'POST'])
def endpointRPA():    
    if request.method == 'POST':
        t0 = time.time()
        try:
            plainCDF = reviewsPlain.copy()
            newName = request.form['newName']
            rating = request.form['rating']
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            tryCount = request.form['tryCount']
            filtered = fil2.main(plainCDF, "", newName, "", "", "")                
            if (filtered.empty):
                return "No data for those filters."  
            filtered.sort_values(by=['date'], ascending=False, inplace=True)  
            status = list()    
            minStart = 20 * (int(tryCount) - 1)
            maxEnd = 20 * int(tryCount) - 1
            filtered = filtered.iloc[minStart:maxEnd]
            if (filtered.empty):
                return "No more reviews to show. Press the button again to view latest reviews!" 
            for i in range(filtered.shape[0]):
                status.append(str(filtered["stars"].iloc[i]))
                status.append(str(filtered["date"].iloc[i]))
                status.append(str(filtered["text"].iloc[i]))
        except:
            return "Python  error: " + traceback.format_exc().splitlines()[-1]
        return jsonify(status)
    else:
        return "Send a proper POST request!"

@app.route("/filter", methods=['GET', 'POST'])
def endpointFilter():    
    if request.method == 'POST':
        try:
            global lastDF
            newName = request.form['newName']  
            dateLow = request.form['dateLow']
            dateUp = request.form['dateUp']
            granulation = request.form['granulation']            
            if str(lastDF["new_name"].iloc[0]) == newName:
                currentDF = lastDF.copy()
                filtered = fil2.main(currentDF, "", "", "", dateLow, dateUp)
                result = woc3.main(filtered, granulation)
                if (filtered.empty):
                    return "No data for those filters."
            else:
                currentDF = allDataLem.copy()
                filtered = fil2.main(currentDF, "", newName, "", dateLow, dateUp)
                result = woc3.main(filtered, granulation)
                if (filtered.empty):
                    return "No data for those filters."
                lastDF = filtered.copy()
        except:
            return "Python  eroor: " + traceback.format_exc().splitlines()[-1]
        return jsonify(filtered.to_json(orient='records'), result.to_json(orient='records'))
    else:
        return "Send a proper POST request!"

@app.route("/monthView")
def endpointMV():    
    return  render_template("/SingleYear.html")
    
if __name__ == "__main__":
    app.run(port=5001, debug=True, use_reloader=False, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jan/2020 16:24:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:24:51] "GET /static/stylesheets/index.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:24:51] "GET /static/stylesheets/PlovdivSans.otf HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:24:52] "GET /AllSample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:24:59] "GET /AllSample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:26:51] "GET /AllSample HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:26:52] "POST /restaurantsNames HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:26:54] "GET /allRestReviews?%22Bacchanal%20Buffet%22%20-%2089109 HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:26:54] "GET /static/stylesheets/index.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:26:54] "GET /static/stylesheets/PlovdivSans.otf HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:26:56] "POST /plainReviewsAll HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2020 16:27:01] "POST /p

In [22]:
reviewsPlain2 = reviewsPlain.copy()
reviewsPlain2

,stars,date,text,new_name
0,5,2017-08-02,Dont miss the peppercorn steak . The peppercor...,"""Mon Ami Gabi"" - 89109"
1,3,2013-03-16,This is a great place to come during the summe...,"""Mon Ami Gabi"" - 89109"
2,4,2013-01-04,Mon Ami Gabi is practically a landmark. In fac...,"""Mon Ami Gabi"" - 89109"
3,4,2010-07-02,Having done Vegas (at least once) every year f...,"""Mon Ami Gabi"" - 89109"
4,5,2017-08-04,"To start with, I'm French and I'm a Vegas loca...","""Mon Ami Gabi"" - 89109"
...,...,...,...,...
3346019,1,2017-08-18,Bar is absolute trash. Came in spend over 150 ...,"""Timbers Rancho"" - 89130"
3346020,4,2017-10-13,Nice place! I am a non-smoker and although a f...,"""Timbers Rancho"" - 89130"
3346021,5,2009-05-22,Are you kidding me?? 1 star? \r\r\r\n\r\r\r\nO...,"""Budweiser Racing Track Bar & Grill"" - 89119"
3346022,2,2013-05-14,$14.35 for a beer and a coke. \r\r\r\n\r\r\r\n...,"""Budweiser Racing Track Bar & Grill"" - 89119"


In [19]:
#reviewsPlain2.iloc[0:24]
status = list();
reviewsPlain2 = reviewsPlain2.iloc[25:49]
for i in range(reviewsPlain2.shape[0]):
    status.append(str(reviewsPlain2["stars"].iloc[i]))
    status.append(str(reviewsPlain2["date"].iloc[i]))
    status.append(str(reviewsPlain2["text"].iloc[i]))
status

[]

In [13]:
reviewsPlain2.sort_values(by=['date'], ascending=False, inplace=True)
reviewsPlain2

,stars,date,text,new_name
3261632,5,2017-12-11,I'd eat here everyday if I could! We come her...,"""Las Islitas"" - 89142"
3076855,3,2017-12-11,"If you are hungry and ready to eat, skip that ...","""Miki's Restaurant"" - 28105"
2576457,4,2017-12-11,Love the food and the service here. Have alway...,"""El Rio Grande"" - 53589"
896344,1,2017-12-11,Worst dim sum I've ever had. Food was not fres...,"""Yong Kang Street"" - 89109"
896243,5,2017-12-11,Great atmosphere and super friendly service. T...,"""Yong Kang Street"" - 89109"
...,...,...,...,...
3051599,4,2004-10-19,Love their pizza. They used to have a great ta...,"""Papa John's Pizza"" - 61820"
2873152,4,2004-10-19,Pokey Sticks are the best!,"""Gumby's Pizza"" - 61820"
3338941,4,2004-10-19,Love their subs. Cheap and top shelf ingredients.,"""Jimmy John's"" - 61820"
976502,4,2004-10-19,Good stuff. Pricey by normal pizza standards.,"""Papa Del's Pizza"" - 61820"


In [2]:
t0 = time.time()
client = MongoClient()
db = client.test_database
collection = db.test_collection

reviewsMongo = db.reviews

allDataLem = pd.DataFrame(list(reviewsMongo.find({})))
del allDataLem["_id"]
allDataLem = allDataLem.astype({'stars': np.int8})
t1 = time.time()
print("Database converted in: " + str(t1-t0))

Database converted in: 91.46134400367737


In [2]:
def readCSV(item):
    data = pd.read_csv(item, dtype={0: int, "business_id":str, "stars":np.int8, "date":str, "text":str, "name":str, "postal_code":str, "ccur":np.int64}, index_col=0, encoding='latin-1')
    
    df = pd.DataFrame(data)
    return df

In [3]:
def initialLoad():
    
    allDataLem = readCSV('static/revSamLem2.csv')
    allDataLem["lemmas"] = allDataLem["lemmas"].apply(literal_eval)
    
    return allDataLem

In [66]:
t0 = time.time()
allDataLem = initialLoad()
t1 = time.time()
print("Time is : " + str(t1-t0))
allDataLem.dtypes

Time is : 1.568802833557129


business_id    object
stars           int64
date           object
text           object
name           object
postal_code     int64
ccur            int64
lemmas         object
dtype: object

In [4]:
t0 = time.time()
allDataLem = initialLoad()
t1 = time.time()
print("Time is : " + str(t1-t0))
allDataLem.dtypes

C:\Users\Bogy\Anaconda3\envs\Envir. Python 3.6\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Time is : 352.08891129493713


business_id    object
stars            int8
date           object
text           object
name           object
postal_code    object
ccur            int64
lemmas         object
dtype: object

In [5]:
zaProbi = allDataLem.copy()
zaProbi

,business_id,stars,date,text,name,postal_code,ccur,lemmas
index,,,,,,,,
5dff854448cc11dacfa82eb3,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-02,Dont miss the peppercorn steak . The peppercor...,"""Mon Ami Gabi""",89109,7328,"[do, miss, peppercorn, steak, peppercorn, slig..."
5dff854448cc11dacfa82eb4,4JNXUYY8wbaaDmk3BPzlWw,3,2013-03-16,This is a great place to come during the summe...,"""Mon Ami Gabi""",89109,7328,"[great, place, come, summer, patio, seat, give..."
5dff854448cc11dacfa82eb5,4JNXUYY8wbaaDmk3BPzlWw,4,2013-01-04,Mon Ami Gabi is practically a landmark. In fac...,"""Mon Ami Gabi""",89109,7328,"[mon, ami, gabi, practically, landmark, fact, ..."
5dff854448cc11dacfa82eb6,4JNXUYY8wbaaDmk3BPzlWw,4,2010-07-02,Having done Vegas (at least once) every year f...,"""Mon Ami Gabi""",89109,7328,"[do, vega, little, every, year, past, year, ni..."
5dff854448cc11dacfa82eb7,4JNXUYY8wbaaDmk3BPzlWw,5,2017-08-04,"To start with, I'm French and I'm a Vegas loca...","""Mon Ami Gabi""",89109,7328,"[start, french, vega, local, time, time, like,..."
...,...,...,...,...,...,...,...,...
5dff855f48cc11dacfdb3d16,nD_7R5bVxRYNHKv_dGlhBA,1,2017-08-18,Bar is absolute trash. Came in spend over 150 ...,"""Timbers Rancho""",89130,11,"[bar, absolute, trash, come, spend, 150, dolla..."
5dff855f48cc11dacfdb3d17,nD_7R5bVxRYNHKv_dGlhBA,4,2017-10-13,Nice place! I am a non-smoker and although a f...,"""Timbers Rancho""",89130,11,"[nice, place, non, smoker, although, folk, enj..."
5dff855f48cc11dacfdb3d18,nCjsK9-_xH0bxuEqdAc8TA,5,2009-05-22,Are you kidding me?? 1 star? \r\r\r\n\r\r\r\nO...,"""Budweiser Racing Track Bar & Grill""",89119,11,"[kid, 1, star, okay, let, say, bar, also, know..."


In [17]:
lastDF = pd.DataFrame(index = ['A'], columns = ['name', 'postal_code'])
lastDF
if str(lastDF["name"].iloc[0]) + " - " + str(lastDF["postal_code"].iloc[0]) == zaProbi.iloc[0, 0]:
    print(str(lastDF["name"].iloc[0]) + " - " + str(lastDF["postal_code"].iloc[0]))
    print(zaProbi.iloc[0, 0])
    print("da")
else:
    print(str(lastDF["name"].iloc[0]) + " - " + str(lastDF["postal_code"].iloc[0]))
    print(zaProbi.iloc[0, 0])
    print("ne")

nan - nan
4JNXUYY8wbaaDmk3BPzlWw
ne


In [8]:
zaProbi.to_csv('revSam2.csv')

In [28]:
stringa1 = """"McDonald'"""
stringa2 = """s"""
stringa3 = stringa1 + stringa2
print(stringa3)
zaProbi = fil2.main(zaProbi, "", stringa3 + '" - M4C 1H9', "", '', '')
zaProbi

"McDonald's


,business_id,stars,date,text,name,postal_code,ccur,lemmas
index,,,,,,,,
4040851,kqW_BKO3XCOx8ifbzQsnGA,3,2011-09-22,Ah micky d's. What's different about this loca...,"""McDonald's""",M4C 1H9,11,"['ah', 'micky', 'different', 'location', 'noth..."
4040858,kqW_BKO3XCOx8ifbzQsnGA,4,2013-10-14,This location actually impressed me. It was ar...,"""McDonald's""",M4C 1H9,11,"['location', 'actually', 'impress', 'around', ..."
4040857,kqW_BKO3XCOx8ifbzQsnGA,4,2014-11-20,This is a great McDonald's location- very quic...,"""McDonald's""",M4C 1H9,11,"['great', 'mcdonald', 'location', 'quick', 'ef..."
4040856,kqW_BKO3XCOx8ifbzQsnGA,3,2014-05-16,"This location is close to my doc's office, and...","""McDonald's""",M4C 1H9,11,"['location', 'close', 'doc', 'office', 'since'..."
4040854,kqW_BKO3XCOx8ifbzQsnGA,3,2017-08-17,I was only here for the 67 cent hamburger extr...,"""McDonald's""",M4C 1H9,11,"['67', 'cent', 'hamburger', 'extravaganza', 'y..."
4040853,kqW_BKO3XCOx8ifbzQsnGA,3,2012-05-09,"This place is open 24 hours, which is helpful ...","""McDonald's""",M4C 1H9,11,"['place', 'open', '24', 'hour', 'helpful', 'ge..."
4040850,kqW_BKO3XCOx8ifbzQsnGA,1,2017-02-05,I give this location 1 star for being open 24h...,"""McDonald's""",M4C 1H9,11,"['give', 'location', '1', 'star', 'open', '24h..."
4040849,kqW_BKO3XCOx8ifbzQsnGA,5,2015-12-03,"I love this McDonald's, the service is great h...","""McDonald's""",M4C 1H9,11,"['love', 'mcdonald', 'service', 'great', 'staf..."
4040848,kqW_BKO3XCOx8ifbzQsnGA,1,2015-01-08,Worst customer service ever! I live nearby and...,"""McDonald's""",M4C 1H9,11,"['wrong', 'customer', 'service', 'ever', 'live..."


In [11]:
zaProbi = fil2.main(zaProbi, "", '"Holsteins Shakes and Buns" - 89109', "", '2010-01-01', '2010-12-31')
filtered0 = zaProbi.copy()
filtered1 = fil2.main(zaProbi, "", '', "2", "", "").copy()
filtered0

,business_id,stars,date,text,name,postal_code,ccur,lemmas
index,,,,,,,,
2154825,P7pxQFqr7yBKMMI2J51udw,3,2010-12-30,Popped into Holsteins on X-mas eve to grab som...,"""Holsteins Shakes and Buns""",89109,2766,"['pop', 'holsteins', 'x', 'ma', 'eve', 'grab',..."
2152817,P7pxQFqr7yBKMMI2J51udw,4,2010-12-23,I went here for lunch on the opening week at C...,"""Holsteins Shakes and Buns""",89109,2766,"['go', 'lunch', 'open', 'week', 'cosmopolitan'..."
2154298,P7pxQFqr7yBKMMI2J51udw,4,2010-12-30,A really interesting menu with some very creat...,"""Holsteins Shakes and Buns""",89109,2766,"['really', 'interest', 'menu', 'create', 'choi..."
2154305,P7pxQFqr7yBKMMI2J51udw,3,2010-12-24,Pros: Holsteins is a nice looking burger joint...,"""Holsteins Shakes and Buns""",89109,2766,"['pro', 'holsteins', 'nice', 'look', 'burger',..."
2154316,P7pxQFqr7yBKMMI2J51udw,2,2010-12-27,this is another place with potential. The menu...,"""Holsteins Shakes and Buns""",89109,2766,"['another', 'place', 'potential', 'menu', 'sou..."
2152636,P7pxQFqr7yBKMMI2J51udw,1,2010-12-20,"After checking out the Cosmo hotel, we went on...","""Holsteins Shakes and Buns""",89109,2766,"['check', 'cosmo', 'hotel', 'go', 'hunt', 'som..."
2152402,P7pxQFqr7yBKMMI2J51udw,4,2010-12-20,Just tried out this place as a buddy and I wer...,"""Holsteins Shakes and Buns""",89109,2766,"['try', 'place', 'buddy', 'check', 'new', 'cos..."
2152538,P7pxQFqr7yBKMMI2J51udw,4,2010-12-18,"Burgers are flavorfull, creative and good size...","""Holsteins Shakes and Buns""",89109,2766,"['burgers', 'flavorfull', 'creative', 'good', ..."
2154906,P7pxQFqr7yBKMMI2J51udw,3,2010-12-31,I would never spend the amount of money at a b...,"""Holsteins Shakes and Buns""",89109,2766,"['would', 'never', 'spend', 'amount', 'money',..."
